# Download the data from the production database

In [3]:
import os

import pandas as pd
import polars as pl
import psycopg2
from dotenv import load_dotenv

In [4]:
load_dotenv()
DATABASE_URL = os.environ['DATABASE_URL']

In [82]:
conn = psycopg2.connect(DATABASE_URL)

In [39]:
user_df = pl.read_database('select * from public.user', conn)
meme_source_df = pl.read_database('select id, type, url, status, language_code, parsed_at, created_at, updated_at, added_by from public.meme_source', conn)
user_language_df = pl.read_database('select * from user_language', conn)

In [ ]:
# ~15-20 secs
meme_df = pl.read_database("select id, meme_source_id, raw_meme_id, status, type, telegram_file_id, caption, language_code, created_at, updated_at, published_at, ocr_result #>> '{}' ocr_result from public.meme", conn)

In [62]:
# ~1.5 min
res = []
dates = pd.date_range(start='2024-01-01', end='2024-05-10', freq='d').strftime('%Y-%m-%d')
for d_from, d_to in zip(dates, dates[1:]):
    res.append(pl.read_database(f"select * from public.user_meme_reaction where sent_at >= '{d_from}'::timestamp and sent_at < '{d_to}'::timestamp", conn))
res = [df for df in res if len(df)]

In [68]:
user_meme_df = pl.concat(res)

In [88]:
user_df.write_parquet('user.pq')
meme_df.write_parquet('meme.pq')
meme_source_df.write_parquet('meme_source.pq')
user_language_df.write_parquet('user_language.pq')

In [89]:
user_meme_df.write_parquet('user_meme_reaction.pq')

In [90]:
conn.close()